# Subir un modelo de ML a la nube

#### Vamos a ver cómo poner disponible un modelo de Machine Learning a la nube para que pueda ser consumido por cualquiera como una API. La nube que usaremos será la de Amazon.


### 1. Repositorio de Github

Vamos a clonar el repositorio donde tenemos el código que vamos a usar

### 2. Creamos una instancia de EC2
Lanzamos una nueva instancia con la siguiente configuración:
- Ubuntu
- Versión 22.04
- 64 bits (por defecto)
- Tipo de instancia t2.micro
- Crear par de claves de tipo RSA y formato .ppk. Se descarga un fichero, guardarlo.
- Activamos las opciones para permitir el tráfico SSH y HTTP/S
- Lanzamos la instancia

Con esto ya tenemos nuestra instancia creada

### 3. Conexión con Putty
Nos descargamos Putty y lo configuramos:
- En Connection -> SSH -> Auth seleccionamos el fichero con las claves que nos hemos descargado
- En Host name, ponemos ubuntu@IP, sustituyendo la IP por la IP pública de tu máquina EC2
- Ponemos un nombre a la configuración para acceder más rápidamente la próxima vez

Ya estamos conectados a nuestra máquina de AWS

### 4. Instalamos el software necesario
Ejecutamos:
- sudo apt-get update
- sudo apt-get install python3-venv

### 5. Descargamos el código
- Creamos directorio nuevo y dentro clonamos el repositorio (mkdir prod)
- Creamos otra carpeta (endpoint) y entramos
- Dentro, creamos un entorno virtual (python3 -m venv venv)
- Instalamos en el venv flask y sklearn
- Creamos los links a los archivos que necesitamos (ln -s ...)
- Comprobamos que funciona

### 6. Gunicorn
- Instalamos el paquete gunicorn con el venv
- Creamos en endpoint una carpeta llamada logs
- Creamos fichero gunicorn.conf.py y le ponemos:

        bind="0.0.0.0:8000"
        workers=1
        timeout=120
        accesslog = "/home/ubuntu/prod/endpoint/logs/gunicorn.access.log"
        errorlog = "/home/ubuntu/prod/endpoint/logs/gunicorn.error.log"
        capture_output = True
        loglevel = "info"

Probamos: /home/ubuntu/prod/endpoint/venv/bin/gunicorn -c /home/ubuntu/prod/endpoint/gunicorn.conf.py app:app

### 7. Gunicorn como servicio
Creamos en /etc/systemd/system un fichero prod.service con este contenido:

    [Unit]
    Description=Gunicorn instance for productivization app
    After=network.target
    [Service]
    User=ubuntu
    Group=ubuntu
    WorkingDirectory=/home/ubuntu/prod/endpoint
    ExecStart=/home/ubuntu/prod/endpoint/venv/bin/gunicorn -c /home/ubuntu/prod/endpoint/gunicorn.conf.py app:app
    Restart=always
    [Install]
    WantedBy=multi-user.target

Arrancamos el servicio:
- sudo systemctl daemon-reload
- sudo systemctl start prod.service
- sudo systemctl enable prod.service

Comprobamos el estado con sudo systemctl status prod
Probamos con curl

### 8. Nginx
Instalamos Nginx: sudo apt-get install nginx

Creamos un archivo en /etc/nginx/sites-available/prod con lo siguiente:

    server {
     listen 80;
     server_name XXXXXXXXX; (LA IP DE LA MAQUINA)
     location / {
      proxy_pass http://127.0.0.1:8000;
     }
    }

Creamos un link del fichero anterior en /etc/nginx/sites-enabled/prod

Arrancamos el servicio:
- sudo systemctl start nginx
- sudo systemctl enable nginx

Comprobamos el servicio: sudo systemctl status nginx

Comprobamos que todo funciona correctamente

In [3]:
import requests

data = requests.get('http://3.19.228.251/api/data1/0/data2/0/data3/0/data4/0')

data.json()

{'prediction': {'name': 'Setosa', 'type': '0'}}

In [ ]:
http://3.19.228.251/api/data1/0/data2/0/data3/0/data4/0